# Flagging days/periods with Snow



In [1]:
# import relevant modules
import configparser
from psycopg2 import connect
import psycopg2.sql as pg
import pandas.io.sql as pandasql
import datetime

# connect to database
CONFIG = configparser.ConfigParser()
CONFIG.read(r'C:\Users\alouis2\Documents\Python Scripts\db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

The following code automates the flagged data creation process through two functions. `formatter` takes in a dictionary who's keys are snow dates and who's values are time periods with snow corresponding to their key. This function returns a string that is used by the `flagged` function that returns a dataframe containing flagged data. 

In [28]:
# run query
january = {datetime.date(2018,1,2):['AM Peak', 'Midday', 'Evening'],
          datetime.date(2018,1,7):['PM Peak'],
          datetime.date(2018,1,8):['AM Peak', 'Midday'],
          datetime.date(2018,1,12):['Midday', 'PM Peak', 'Evening'],
          datetime.date(2018,1,15):['Midday', 'PM Peak', 'Evening'],
          datetime.date(2018,1,16):['AM Peak', 'Evening'],
          datetime.date(2018,1,29):['Midday', 'PM Peak', 'Evening'],
          datetime.date(2018,1,30):['AM Peak'],
          datetime.date(2018,1,31):['AM Peak', 'Midday']}

def flagged(x):
    def formatter(x):
        string = ''' dt = '%s' '''
        for i in range(len(x)-1):
            string = string + '''or dt = '%s' '''
        string = string % tuple([key for key in x])
        return  '''select pilot.dt, pilot.street, pilot.direction, pilot.period, 
                    pilot.tt as pilot, baseline.tt as baseline 
                    from king_pilot.dash_daily_dev as pilot,  king_pilot.dash_baseline_dev as baseline 
                    where (%s)
                    and category = 'Pilot' and  pilot.day_type = 'Weekday' and pilot.street = baseline.street 
                    and pilot.direction = baseline.direction and pilot.day_type = baseline.day_type 
                    and pilot.period = baseline.period order by dt, street, direction, period''' % string
    string = formatter(x)
    sql = pg.SQL(string)
    snow = pandasql.read_sql(sql, con)
    change = [snow.pilot.values[i]-snow.baseline.values[i] for i in range(len(snow))]
    snow['change'] = change
    flag = [0 for i in range(len(snow))]
    snow['snow_flag'] = flag 
    for key in x:
        for i in x[key]:
            for j in range(len(snow)):
                if snow.dt.values[j] == key and snow.period.values[j] == i:
                    snow.snow_flag.values[j] += 1        
    return snow



In [29]:
flagged(january)

,dt,street,direction,period,pilot,baseline,change,snow_flag
0,2018-01-02,Bathurst,Northbound,AM Peak,4.264326,5.298643,-1.034317,1
1,2018-01-02,Bathurst,Northbound,Evening,4.163933,5.412162,-1.248228,1
2,2018-01-02,Bathurst,Northbound,Midday,4.595783,5.607554,-1.011770,1
3,2018-01-02,Bathurst,Northbound,PM Peak,4.919735,6.108102,-1.188367,0
4,2018-01-02,Bathurst,Southbound,AM Peak,4.059722,5.109653,-1.049930,1
5,2018-01-02,Bathurst,Southbound,Evening,3.824661,5.248978,-1.424317,1
6,2018-01-02,Bathurst,Southbound,Midday,3.896654,4.890124,-0.993471,1
7,2018-01-02,Bathurst,Southbound,PM Peak,4.358040,6.973702,-2.615662,0
8,2018-01-02,Dundas,Eastbound,AM Peak,8.537553,10.599406,-2.061853,1
9,2018-01-02,Dundas,Eastbound,Evening,10.367028,11.712298,-1.345270,1
